In [3]:
import os
import requests
import boto3

# Configuration
MAPBOX_USERNAME = 'shuyangwang'
MAPBOX_ACCESS_TOKEN = 'sk.eyJ1Ijoic2h1eWFuZ3dhbmciLCJhIjoiY20xZHY2ZmwxMnE0cjJqcHNrNmd1M2YwMiJ9.9GvM0blOBJmCVlpLE-ABYw'
TIFF_FILE_PATH = '/Users/shuyang/Data/DTM/LakeNipissing-DTM-D/hillshade/Projected/merged_output_optimized.tif'  # Path to your TIFF file
TILESET_NAME = 'LakeNipissing-DTM-C_Optimized'  # Custom name for your tileset

In [4]:


# Step 1: Retrieve temporary credentials
def get_s3_credentials():
    print("Retrieving S3 credentials...")
    url = f'https://api.mapbox.com/uploads/v1/{MAPBOX_USERNAME}/credentials?access_token={MAPBOX_ACCESS_TOKEN}'
    response = requests.post(url)
    
    if response.status_code != 200:
        print("Error retrieving credentials:", response.text)
        response.raise_for_status()
    
    print("Credentials retrieved successfully.")
    return response.json()

# Step 2: Stage the file on Amazon S3
def stage_file_on_s3(credentials, file_path):
    print("Staging file on S3...")
    s3 = boto3.client(
        's3',
        aws_access_key_id=credentials['accessKeyId'],
        aws_secret_access_key=credentials['secretAccessKey'],
        aws_session_token=credentials['sessionToken'],
    )
    
    bucket = credentials['bucket']
    key = credentials['key']
    
    try:
        s3.upload_file(Filename=file_path, Bucket=bucket, Key=key)
        print(f'File staged in bucket: {bucket}, key: {key}')
    except Exception as e:
        print("Error uploading file to S3:", str(e))
        raise
    
    return bucket, key

# Step 3: Create an upload in Mapbox
def create_upload(bucket, key, tileset_name):
    print("Creating upload in Mapbox...")
    url = f'https://api.mapbox.com/uploads/v1/{MAPBOX_USERNAME}?access_token={MAPBOX_ACCESS_TOKEN}'
    payload = {
        'url': f'http://{bucket}.s3.amazonaws.com/{key}',
        'tileset': f'{MAPBOX_USERNAME}.{tileset_name}',  # Use the custom tileset name
        'name': tileset_name  # Use the custom name
    }
    
    response = requests.post(url, json=payload)
    
    if response.status_code != 201:
        print("Error creating upload:", response.text)
        response.raise_for_status()
    
    print("Upload created successfully.")
    return response.json()

# Execute the upload process
try:
    # Get temporary credentials
    credentials = get_s3_credentials()

    # Set AWS environment variables
    os.environ['AWS_ACCESS_KEY_ID'] = credentials['accessKeyId']
    os.environ['AWS_SECRET_ACCESS_KEY'] = credentials['secretAccessKey']
    os.environ['AWS_SESSION_TOKEN'] = credentials['sessionToken']

    # Stage the file
    bucket, key = stage_file_on_s3(credentials, TIFF_FILE_PATH)

    # Create the upload with a custom tileset name
    upload_response = create_upload(bucket, key, TILESET_NAME)
    print('Upload initiated:', upload_response)

except Exception as e:
    print("An error occurred:", str(e))


Retrieving S3 credentials...
Credentials retrieved successfully.
Staging file on S3...
File staged in bucket: tilestream-tilesets-production, key: 2f/_pending/bf76calkjqm1b1s5mxgmgf1mc/shuyangwang
Creating upload in Mapbox...
Error creating upload: {"message":"An upload is already active"}
An error occurred: 422 Client Error: Unprocessable Entity for url: https://api.mapbox.com/uploads/v1/shuyangwang?access_token=sk.eyJ1Ijoic2h1eWFuZ3dhbmciLCJhIjoiY20xZHY2ZmwxMnE0cjJqcHNrNmd1M2YwMiJ9.9GvM0blOBJmCVlpLE-ABYw
